In [62]:
!pip install vaderSentiment langdetect emoji nltk vaderSentiment

StatementMeta(spark0623, 19, 63, Finished, Available)

In [63]:
from pyspark.sql.functions import udf,monotonically_increasing_id
from pyspark.sql.types import StringType, FloatType
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from pyspark.sql import functions as f  
import requests
import pandas as pd 
import datetime
import emoji
import nltk
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from notebookutils import mssparkutils

StatementMeta(spark0623, 19, 64, Finished, Available)

In [66]:
nltk.download('punkt')
nltk.download('stopwords')

StatementMeta(spark0623, 19, 67, Finished, Available)

[nltk_data] Downloading package punkt to /home/trusted-service-
[nltk_data]     user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/trusted-service-
[nltk_data]     user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [44]:
reddit_df = spark.read.parquet('abfss://files@datalakey4zmcph.dfs.core.windows.net/synapse/workspaces/data/reddit_cms/')

StatementMeta(spark0623, 19, 45, Finished, Available)

In [112]:
reddit_df.show()

StatementMeta(spark0623, 19, 113, Finished, Available)

+-------------------+--------------------+------+-----------+
|        created_utc|             comment|symbol|collectedAt|
+-------------------+--------------------+------+-----------+
|2024-05-29 16:27:25|Buy a few that ta...|  NVDA| 2024-06-23|
|2024-05-29 16:27:00|Buy VT and dont w...|  NVDA| 2024-06-23|
|2024-05-29 16:26:18|You and me both, ...|  NVDA| 2024-06-23|
|2024-05-29 16:26:18|There's a reason ...|  NVDA| 2024-06-23|
|2024-05-29 16:22:20|Your buying perce...|  NVDA| 2024-06-23|
|2024-05-29 16:20:18|Just comfort your...|  NVDA| 2024-06-23|
|2024-05-29 16:18:15|as long as that m...|  NVDA| 2024-06-23|
|2024-05-29 16:14:46|Simple rule.\nDon...|  NVDA| 2024-06-23|
|2024-05-29 16:11:39|You sold for a re...|  NVDA| 2024-06-23|
|2024-05-29 16:11:07|Try and hold for ...|  NVDA| 2024-06-23|
|2024-05-29 16:03:39|Just buy into ano...|  NVDA| 2024-06-23|
|2024-05-29 16:01:55|Another thing to ...|  NVDA| 2024-06-23|
|2024-05-29 16:01:38|Not the worst but...|  NVDA| 2024-06-23|
|2024-05

In [113]:
reddit_df_v1 = (
    reddit_df.withColumn('wordCount', f.size(f.split(f.col('comment'), ' ')))
    .filter(f.col('wordCount')>5)
    .withColumn("id", monotonically_increasing_id())
    .select('created_utc','id','comment','symbol')
    ).toPandas()

StatementMeta(spark0623, 19, 114, Finished, Available)

In [115]:
# remove non english word
def is_english(text):
    try:
        return detect(text) == 'en'
    except LangDetectException:
        return False
    
# change text into lowercase
def to_lowercase(text):
    return text.lower()

# remove punctuation
def rm_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def tokenise(text):
    return word_tokenize(text)

# remove stop words 
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [word for word in tokens if word not in stop_words]

# rejoin tokens
def rejoin_tokens(tokens):
    return ' '.join(tokens)

# main function for preprocess_text 
def preprocess_text(text):
    text = to_lowercase(text)
    text = rm_punctuation(text)
    tokens = tokenise(text)
    tokens = remove_stopwords(tokens)
    return rejoin_tokens(tokens)

StatementMeta(spark0623, 19, 116, Finished, Available)

In [116]:
reddit_df_v1['is_english'] = reddit_df_v1['comment'].apply(is_english)
reddit_df_v2 = reddit_df_v1[reddit_df_v1['is_english']]
reddit_df_v2['preprocessed_text'] = reddit_df_v2['comment'].apply(preprocess_text)

StatementMeta(spark0623, 19, 117, Finished, Available)

/tmp/ipykernel_78401/1873903993.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit_df_v2['preprocessed_text'] = reddit_df_v2['comment'].apply(preprocess_text)


In [117]:
def is_string(text):
    return isinstance(text, str)

reddit_df_v2['is_string']=reddit_df_v2['preprocessed_text'].apply(is_string)
reddit_df_v3 = reddit_df_v2[reddit_df_v2['is_string']]

StatementMeta(spark0623, 19, 118, Finished, Available)

/tmp/ipykernel_78401/136230020.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit_df_v2['is_string']=reddit_df_v2['preprocessed_text'].apply(is_string)


In [118]:
def analyze_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(text)
    return scores['compound']

def categorize_sentiment(compound_score):
    if compound_score >= 0.05:
        return 'positive'
    elif compound_score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

StatementMeta(spark0623, 19, 119, Finished, Available)

In [119]:
reddit_df_v3['sentiment'] = reddit_df_v3['preprocessed_text'].apply(analyze_sentiment)
reddit_df_v3['sentiment_category'] = reddit_df_v3['sentiment'].apply(categorize_sentiment)

StatementMeta(spark0623, 19, 120, Finished, Available)

In [120]:
reddit_df_v4 = reddit_df_v3[['id','created_utc','sentiment','sentiment_category']]

StatementMeta(spark0623, 19, 121, Finished, Available)

In [121]:
final_df = pd.merge(reddit_df_v1,reddit_df_v4, left_on='id', right_on='id', how='left')

StatementMeta(spark0623, 19, 122, Finished, Available)

In [124]:
result = final_df[['created_utc_x','symbol','comment','sentiment','sentiment_category']]

StatementMeta(spark0623, 19, 125, Finished, Available)

In [127]:
spdf = spark.createDataFrame(result)
datalake_nm = 'datalake'+mssparkutils.env.getWorkspaceName()[7:] # get datalake name 
file_path ='abfss://files@{0}.dfs.core.windows.net/synapse/workspaces/data/reddit_sentiments'.format(datalake_nm)
spdf.write.parquet(file_path,mode='overwrite')

StatementMeta(spark0623, 19, 128, Finished, Available)